In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)

/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)


In [3]:
!unzip GeorgiaTech.zip

Archive:  GeorgiaTech.zip
replace gt_db_train/s01/01.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: An
  inflating: gt_db_train/s01/01.jpg  
  inflating: gt_db_train/s01/02.jpg  
  inflating: gt_db_train/s01/03.jpg  
  inflating: gt_db_train/s01/04.jpg  
  inflating: gt_db_train/s01/05.jpg  
  inflating: gt_db_train/s01/06.jpg  
  inflating: gt_db_train/s01/07.jpg  
  inflating: gt_db_train/s01/08.jpg  
  inflating: gt_db_train/s01/09.jpg  
  inflating: gt_db_train/s01/10.jpg  
  inflating: gt_db_train/s02/01.jpg  
  inflating: gt_db_train/s02/02.jpg  
  inflating: gt_db_train/s02/03.jpg  
  inflating: gt_db_train/s02/04.jpg  
  inflating: gt_db_train/s02/05.jpg  
  inflating: gt_db_train/s02/06.jpg  
  inflating: gt_db_train/s02/07.jpg  
  inflating: gt_db_train/s02/08.jpg  
  inflating: gt_db_train/s02/09.jpg  
  inflating: gt_db_train/s02/10.jpg  
  inflating: gt_db_train/s03/01.jpg  
  inflating: gt_db_train/s03/02.jpg  
  inflating: gt_db_train/s03/03.jpg  
  inflating: gt_db_train/s0

In [4]:
!pip install keras

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# Create a data generator
from keras.src.legacy.preprocessing.image import ImageDataGenerator

img_rows, img_cols = 100, 100 # image size
input_shape = (img_rows, img_cols, 3) # input shape, 3 represents the number of color channels (RGB)
num_classes=20 # number of classes
train_datagen = ImageDataGenerator(rescale=1./255) # normalize RGB values to [0,1] by dividing each value with 255
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_train',target_size=(img_rows,img_cols),batch_size=10,class_mode='categorical', shuffle=True)
step_size_train=train_generator.n/train_generator.batch_size

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_val',target_size=(img_rows,img_cols),batch_size=10,class_mode='categorical', shuffle=True)
step_size_val=val_generator.n/val_generator.batch_size

Found 200 images belonging to 20 classes.
Found 100 images belonging to 20 classes.


In [6]:
# Initialize a sequential model, which allows us to build the model layer by layer
model = Sequential()

# Add the first convolutional layer.
# 32 indicates the number of filters (kernels) in the layer. Each filter learns to detect different features in the input image.
# kernel_size=(3, 3) specifies the size of the convolutional filters (3x3 pixels).
# activation='relu' applies the ReLU activation function to the output of the convolutional layer, introducing non-linearity.
# input_shape=input_shape sets the shape of the input images expected by the model (height, width, color channels).
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Add the second convolutional layer, similar to the first but with 64 filters.
# Increasing the number of filters allows the model to learn more complex features.
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add a max pooling layer.
# Specifies the size of the pooling window (2x2 pixels).
# Max pooling reduces the spatial dimensions of the feature maps, helping to make the model more robust to variations in the input images.
# Basically return the maximum value of each pool
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer into a 1D layer.
# This prepares the data for the subsequent fully connected (dense) layers.
model.add(Flatten())

# Add a dense (fully connected) layer.
# 128 specifies the number of neurons (units) in the layer.
# activation='relu' applies the ReLU activation function to introduce non-linearity.
model.add(Dense(128, activation='relu'))

# Add a dense (fully connected) layer for classification.
# num_classes specifies the number of output neurons, corresponding to the number of classes to predict.
# Applies the softmax activation function to the output, producing a [0,1] probability distribution over the classes.
model.add(Dense(num_classes, activation='softmax'))

# Print a summary of the model architecture, showing the layers, output shapes, and number of parameters.
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 98, 98, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 96, 96, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 147456)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      18,874,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │           2,580 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,896,468 (72.08 MB)

 Trainable params: 18,896,468 (72.08 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.optimizers import SGD, Adam

# "Learning Rate" = a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated
adam = Adam(learning_rate=0.0003)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy']) # we only need to read the accuracy metric for this
history = model.fit(train_generator,
                   epochs=5,
                   validation_data=val_generator)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 881ms/step - accuracy: 0.9742 - loss: 0.1340 - val_accuracy: 0.9900 - val_loss: 0.1190
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9961 - loss: 0.0498 - val_accuracy: 1.0000 - val_loss: 0.0209
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 909ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accuracy: 1.0000 - val_loss: 0.0074
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 864ms/step - accuracy: 1.0000 - loss: 8.4988e-04 - val_accuracy: 1.0000 - val_loss: 0.0057
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 876ms/step - accuracy: 1.0000 - loss: 4.7424e-04 - val_accuracy: 1.0000 - val_loss: 0.0058


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Plot training accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validate'], loc='upper left')
plt.savefig('Accuracy.png')
plt.show()

# Plot training loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validate'], loc='upper left')
plt.savefig('Loss.png')
plt.show()

In [9]:
import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from skimage.transform import resize

# Accommodate a single color image with dimensions img_rows x img_cols.
X_test = np.zeros((1, img_rows, img_cols, 3), dtype=np.float32)
# Load image from path
img_test = load_img("/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_val/s02/11.jpg")
# Convert the image to array
x_img_test = img_to_array(img_test)
# Resize the image to match the input size expected by the model.
# The 'constant' mode fills new pixels with a constant value, and preserve_range ensures that the pixel values remain within the original range.
x_img_test = resize(x_img_test, (img_rows, img_cols, 3), mode = 'constant', preserve_range = True)
# Normalize pixel values to be between 0 and 1
X_test[0] = x_img_test/255.0
# Make the model predict
preds_test = model.predict(X_test, verbose=1)
print(preds_test)

arr = np.array(preds_test) # Get the prediction model array values (20 values, because 20 classes)
max_index = np.argmax(arr) # Select the value with the highest probability
print(max_index+1) # Get its 1-based index

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[3.8387177e-03 9.1896868e-01 5.7942872e-03 7.0919399e-04 5.2132999e-04
  3.1417378e-04 3.1397420e-03 2.1239258e-03 2.8914770e-03 7.9512852e-04
  9.0570292e-03 2.8835628e-03 3.4097992e-04 2.1839379e-04 1.2017869e-03
  4.6396382e-02 2.5253557e-04 1.3463861e-04 8.0167694e-05 3.3800548e-04]]
2


In [10]:
model.save("/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_model/model_faceID.keras")

In [ ]:
import tensorflow as tf
face_Model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_model/model_faceID.h5')
face_Model.summary()

In [ ]:
# Reload the model to check its accuracy

import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from skimage.transform import resize

X_test = np.zeros((1, img_rows, img_cols, 3), dtype=np.float32)
img_test = load_img("/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_val/s02/13.jpg")
x_img_test = img_to_array(img_test)
x_img_test = resize(x_img_test, (img_rows, img_cols, 3), mode = 'constant', preserve_range = True)
X_test[0] = x_img_test/255.0
preds_test = face_Model.predict(X_test, verbose=1)
print(preds_test)

import numpy as np
arr = np.array(preds_test)
max_index = np.argmax(arr)
print(max_index + 1)

# Data Pre-Augmentation (for `train` only)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# Create a data generator
from keras.src.legacy.preprocessing.image import ImageDataGenerator

img_rows, img_cols = 100, 100 # image size
input_shape = (img_rows, img_cols, 3) # input shape, 3 represents the number of color channels (RGB)
num_classes=20 # number of classes
# Add data augmentation (apart from rescale)
train_datagen = ImageDataGenerator(rescale=1./255, vertical_flip = True, zoom_range = 0.5, rotation_range = 15)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_train',target_size=(img_rows,img_cols),batch_size=10,class_mode='categorical')
step_size_train=train_generator.n/train_generator.batch_size

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_val',target_size=(img_rows,img_cols),batch_size=10,class_mode='categorical', shuffle=True)
step_size_val=val_generator.n/val_generator.batch_size

In [ ]:
# Initialize a sequential model, which allows us to build the model layer by layer
model = Sequential()

# Add the first convolutional layer.
# 32 indicates the number of filters (kernels) in the layer. Each filter learns to detect different features in the input image.
# kernel_size=(3, 3) specifies the size of the convolutional filters (3x3 pixels).
# activation='relu' applies the ReLU activation function to the output of the convolutional layer, introducing non-linearity.
# input_shape=input_shape sets the shape of the input images expected by the model (height, width, color channels).
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Add the second convolutional layer, similar to the first but with 64 filters.
# Increasing the number of filters allows the model to learn more complex features.
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add a max pooling layer.
# Specifies the size of the pooling window (2x2 pixels).
# Max pooling reduces the spatial dimensions of the feature maps, helping to make the model more robust to variations in the input images.
# Basically return the maximum value of each pool
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer into a 1D layer.
# This prepares the data for the subsequent fully connected (dense) layers.
model.add(Flatten())

# Add a dense (fully connected) layer.
# 128 specifies the number of neurons (units) in the layer.
# activation='relu' applies the ReLU activation function to introduce non-linearity.
model.add(Dense(128, activation='relu'))

# Add a dense (fully connected) layer for classification.
# num_classes specifies the number of output neurons, corresponding to the number of classes to predict.
# Applies the softmax activation function to the output, producing a [0,1] probability distribution over the classes.
model.add(Dense(num_classes, activation='softmax'))

# Print a summary of the model architecture, showing the layers, output shapes, and number of parameters.
model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam

# "Learning Rate" = a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated
adam = Adam(learning_rate=0.0003)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy']) # we only need to read the accuracy metric for this
history = model.fit(train_generator,
                   epochs=10,
                   validation_data=val_generator)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Plot training accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy (with data pre-augmentation)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validate'], loc='upper left')
plt.savefig('Accuracy.png')
plt.show()

# Plot training loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validate'], loc='upper left')
plt.savefig('Loss.png')
plt.show()